# DeepAsr (DeepAsrNetwork1)

In [ ]:
# !wget http://www.openslr.org/resources/12/train-clean-100.tar.gz
!wget http://www.openslr.org/resources/12/test-clean.tar.gz

--2022-05-12 10:42:31--  http://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.openslr.org/resources/12/test-clean.tar.gz [following]
--2022-05-12 10:42:31--  https://us.openslr.org/resources/12/test-clean.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘test-clean.tar.gz.2’

test-clean.tar.gz.2 100%[===================>] 330.60M  24.1MB/s    in 15s     

2022-05-12 10:42:47 (21.8 MB/s) - ‘test-clean.tar.gz.2’ saved [346663984/346663984]



In [ ]:
# !tar xzvf train-clean-100.tar.gz
!tar xzvf test-clean.tar.gz

LibriSpeech/LICENSE.TXT
LibriSpeech/README.TXT
LibriSpeech/CHAPTERS.TXT
LibriSpeech/SPEAKERS.TXT
LibriSpeech/BOOKS.TXT
LibriSpeech/test-clean/
LibriSpeech/test-clean/6930/
LibriSpeech/test-clean/6930/75918/
LibriSpeech/test-clean/6930/75918/6930-75918-0000.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0007.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0006.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0005.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0004.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0002.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0011.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0015.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0003.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0010.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0001.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0018.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0008.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0017.flac
LibriSpeech/test-clean/

In [ ]:
! pip install tensorflow

In [ ]:
! pip install deepasr

# 1. Prepare DataSet

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import deepasr as asr
import librosa

In [ ]:
tf.__version__

'2.1.0'

In [ ]:
asr.__version__

'0.1.1'

In [ ]:
np.__version__

'1.19.5'

In [ ]:
# get audios and transcripts
#org_path = './LibriSpeech/train-clean-100/'
org_path = './LibriSpeech/test-clean/'
count = 0
inp = []
k=0
audio_name = []
audio_trans = []
for dir1 in os.listdir(org_path):
  dir2_path = org_path+dir1+'/'
  #print(dir2_path)
  for dir2 in os.listdir(dir2_path):
    dir3_path = dir2_path+dir2+'/'
    
    for audio in os.listdir(dir3_path):
      if audio.endswith('.txt'):
        k+=1
        file_path = dir3_path + audio
        with open(file_path) as f:
          line = f.readlines()
          for lines in line:
            flac_path = dir3_path+lines.split()[0]+'.flac'
            
            audio_name.append(flac_path)

            # print(cmd)
            words2 = lines.split()[1:]
            words4=' '.join(words2)
            audio_trans.append(words4)

In [ ]:
# create dataset
df = pd.DataFrame({"path":audio_name,"transcripts":audio_trans})

In [ ]:
df.shape

(2620, 2)

In [ ]:
# filter transcript less than 100 charcters
train_data = df[df['transcripts'].str.len() < 100]
# train_df = df.sample(n = 3000) 

In [ ]:
train_data.shape

(1512, 2)

# 2. Prepare DeepAsr CTC Pipeline

In [ ]:
# get CTCPipeline
def get_config(feature_type: str = 'spectrogram', multi_gpu: bool = False):
    # audio feature extractor
    features_extractor = asr.features.preprocess(feature_type=feature_type, features_num=161,
                                                 samplerate=16000,
                                                 winlen=0.02,
                                                 winstep=0.025,
                                                 winfunc=np.hanning)
    
    # input label encoder
    alphabet_en = asr.vocab.Alphabet(lang='en')
    # training model
    #model = asr.model.get_deepasrnetwork1(
    model = asr.model.get_deepspeech2(   
        input_dim=161,
        output_dim=29,
        is_mixed_precision=True
    )
    # model optimizer
    optimizer = tf.keras.optimizers.Adam(
        lr=1e-4,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-8
    )
    # output label deocder
    decoder = asr.decoder.GreedyDecoder()
    # CTCPipeline
    pipeline = asr.pipeline.ctc_pipeline.CTCPipeline(
        alphabet=alphabet_en, features_extractor=features_extractor, model=model, optimizer=optimizer, decoder=decoder,
        sample_rate=16000, mono=True, multi_gpu=multi_gpu
    )
    return pipeline

In [ ]:
# CTCPiline for asr
pipeline = get_config(feature_type='spectrogram', multi_gpu=False)

# 3. Model traning

In [ ]:
# train asr model
history = pipeline.fit(train_dataset = train_data, batch_size=8, epochs=1)

# history = pipeline.fit_iter(train_dataset = train_data, batch_size=32, epochs=3,iter_num=500,checkpoint=project_path+'checkpoints')
# history = pipeline.fit_generator(train_dataset = train_data, batch_size=32, epochs=500)

Model: "DeepAsr"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          [(None, None, 161)]  0                                            
__________________________________________________________________________________________________
BN_1 (BatchNormalization)       (None, None, 161)    644         the_input[0][0]                  
__________________________________________________________________________________________________
Conv1D_1 (Conv1D)               (None, None, 512)    412672      BN_1[0][0]                       
__________________________________________________________________________________________________
CBN_1 (BatchNormalization)      (None, None, 512)    2048        Conv1D_1[0][0]                   
____________________________________________________________________________________________

In [ ]:
# save deepasr ctc pipeline
pipeline.save("./")

# 4. Model testing

In [ ]:
pip install h5py==2.10.0

In [ ]:
# load saved ctc pipeline
pipeline1 = asr.pipeline.load("./")

In [ ]:
# get testing audio and transcript from dataset
index = np.random.randint(train_data.shape[0])
data = train_data.iloc[index]
test_file = data[0]
test_transcript = data[1]
# Audio file
print("Audio File:",test_file)
# ground truth
print("Audio Transcription:", test_transcript)
print("Transcript length:",len(test_transcript))

Audio File: ./LibriSpeech/test-clean/4992/23283/4992-23283-0007.flac
Audio Transcription: TO ASK ANY MORE QUESTIONS OF YOU I BELIEVE WOULD BE UNFAIR
Transcript length: 58


In [ ]:
# predict labels
pred= pipeline1.predict(test_file)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [ ]:
pred[0].upper()

'WENT THROUGH THE PLAINS BUT WHEN THEY CAME NEAR THE MOUNTAINS'